In [1]:
!pip install numpy pyarrow fastparquet scipy scikit-learn

In [2]:
import numpy, scipy, pandas as pd, os
from sklearn.feature_extraction import DictVectorizer

In [3]:
# JAN 
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# FEB
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

In [4]:
df_jan = pd.read_parquet('1_data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('1_data/yellow_tripdata_2023-02.parquet')

In [5]:
df_jan


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [6]:
df_jan.shape

(3066766, 19)

In [7]:
df_jan.shape

(3066766, 19)

In [8]:
df_jan.tpep_pickup_datetime = pd.to_datetime(df_jan.tpep_pickup_datetime)
df_jan.tpep_dropoff_datetime = pd.to_datetime(df_jan.tpep_dropoff_datetime)
# tpep_dropoff_datetime

In [9]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

In [10]:
df_jan.duration.describe()

count                   3066766
mean     0 days 00:15:40.139710
std      0 days 00:42:35.661074
min           -1 days +23:30:48
25%             0 days 00:07:07
50%             0 days 00:11:31
75%             0 days 00:18:18
max             6 days 23:09:11
Name: duration, dtype: object

In [11]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration >= 60)]


In [13]:
df_jan.duration.describe()

count    24301.000000
mean       218.135302
std        418.515838
min         60.000000
25%         63.450000
50%         68.450000
75%         79.583333
max      10029.183333
Name: duration, dtype: float64

In [14]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration >= 60)]


In [30]:
df_jan.duration.describe()

count    24301.000000
mean       218.135302
std        418.515838
min         60.000000
25%         63.450000
50%         68.450000
75%         79.583333
max      10029.183333
Name: duration, dtype: float64

In [16]:
# FEATURES

# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

In [38]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

In [39]:
df_jan[categorical].dtypes


PULocationID    object
DOLocationID    object
dtype: object

In [40]:
df_jan[categorical] = df_jan[categorical].astype(str)

In [41]:
df_jan[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [42]:
# df_jan[numerical].dtypes

In [43]:
dv = DictVectorizer()

In [44]:
# train_dicts = df_jan[categorical + numerical].to_dict(orient='records')
train_dicts = df_jan[categorical].to_dict(orient='records')

In [45]:
x_train = dv.fit_transform(train_dicts)

In [46]:
x_train

<24301x485 sparse matrix of type '<class 'numpy.float64'>'
	with 48602 stored elements in Compressed Sparse Row format>

In [47]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=105',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',

In [27]:
target = 'duration'

In [28]:
y_train = df_jan[target].values

In [29]:
y_train

array([  68.1       , 1421.1       , 1376.76666667, ...,   64.13333333,
         66.78333333,   73.        ])